In [6]:
from bs4 import BeautifulSoup
from lxml import etree
import json
import re
import multiprocessing
# import xlwt
import time
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import requests
import logging

## get

In [7]:
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s: %(message)s')

INDEX_URL = 'https://spa1.scrape.center/api/movie/?limit={limit}&offset={offset}'
# https://spa1.scrape.center/api/movie/?limit=10&offset=10

In [8]:
def scrape_api(url):
    logging.info('scraping %s...', url)
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        logging.error('get invalid status code %s while scraping %s', response.status_code, url)
    except requests.RequestException:
        logging.error('error occurred while scraping %s', url, exc_info=True)

In [9]:
LIMIT = 10


def scrape_index(page):
    url = INDEX_URL.format(limit=LIMIT, offset=LIMIT * (page - 1))
    return scrape_api(url)

In [10]:
# 单部电影详情页
# 'https://spa1.scrape.center/api/movie/39/'

DETAIL_URL = 'https://spa1.scrape.center/api/movie/{id}'


def scrape_detail(id):
    url = DETAIL_URL.format(id=id)
    return scrape_api(url)

In [12]:
TOTAL_PAGE = 1


def main():
    for page in range(1, TOTAL_PAGE + 1):
        index_data = scrape_index(page)
        for item in index_data.get('results'):
            id = item.get('id')
            detail_data = scrape_detail(id)
            logging.info('detail data %s', detail_data)


if __name__ == '__main__':
    main()

2022-09-25 10:53:26,569 - INFO: scraping https://spa1.scrape.center/api/movie/?limit=10&offset=0...
2022-09-25 10:53:27,923 - INFO: scraping https://spa1.scrape.center/api/movie/1...
2022-09-25 10:53:30,311 - INFO: detail data {'id': 1, 'name': '霸王别姬', 'alias': 'Farewell My Concubine', 'cover': 'https://p0.meituan.net/movie/ce4da3e03e655b5b88ed31b5cd7896cf62472.jpg@464w_644h_1e_1c', 'categories': ['剧情', '爱情'], 'regions': ['中国内地', '中国香港'], 'actors': [{'name': '张国荣', 'role': '程蝶衣', 'image': 'https://p0.meituan.net/movie/5de69a492dcbd3f4b014503d4e95d46c28837.jpg@128w_170h_1e_1c'}, {'name': '张丰毅', 'role': '段小楼', 'image': 'https://p0.meituan.net/movie/35e74707f69da838d7ba3422b8f6579840705.jpg@128w_170h_1e_1c'}, {'name': '巩俐', 'role': '菊仙', 'image': 'https://p1.meituan.net/moviemachine/b650dcb00c40356934a275515217850f191104.jpg@128w_170h_1e_1c'}, {'name': '吕齐', 'role': '关师傅', 'image': 'https://p1.meituan.net/movie/30e12a78b5e61916edb1e33ce6fec19b34794.jpg@128w_170h_1e_1c'}, {'name': '英达', 'r

## post

In [23]:
kfc = 'http://www.kfc.com.cn/kfccda/ashx/GetStoreList.ashx?op=keyword'

header = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.116 Safari/537.36',
    #'cookie': 'ai_user=5YqS2gz3MaBT/lERkqpo0o|2022-06-26T15:26:26.666Z; _ga=GA1.2.2000435899.1656257187; _gcl_au=1.1.1020666982.1664034236; _gid=GA1.2.265423349.1664034237; __cf_bm=GA5S_sB0elMcNrpbsALXyt7WxomMn_W6VUjA4HCzub0-1664093813-0-AXqQvnwPewLxq64/dVy20XtMqYSJ0vPZp6EQSG5LRMFIkOqF6ccxReWpxwuObyuCi72FnSFgFyDLKWiRl9uPN6Gub0CCvsFZnTr3i1oycRY9ZpF+p4LE6vzR9cLmY8Miaw==; XSRF-TOKEN=eyJpdiI6IlVXUFU3THQ5N2VXd3J4NFUxQ2FBRnc9PSIsInZhbHVlIjoiU2w4MG1ONjVEL2FFVEJPb2FSVlpOdE5QRWlUeGovWGwvODd6QU9Md2RqN00vSi9rSFF6LzRVT1RQc2xvTTN1V0NVdXdOQllubkdqZ2c1OEhRN1hqOStmVEoyT0owalFCRGkzMjFqUTBENWNJd09Ya1dMMVBiamRrS1FvR0ZWVVciLCJtYWMiOiI2NTUzMzAwOTc3ZjZiODc0ODk5NTUzYWYyZWJmMTI2OWE3ZGI3MDY5ODVjYjBkNzRhOTU3NjNkNTIwMGJmNTI5In0=; laravel_session=eyJpdiI6Im9wM2t1Q0luRnFqVEhKZDBZZkhIMmc9PSIsInZhbHVlIjoiUlBOVmZZbnZ4aUpBcXFiYWdqL0lYYjlrVTh1QjR2SWtkY2w2TW5XY0RWeStpVm5RSXd4VmRMK3hHVldTNzV0ZTZEbjZNMm5TL1ZHdkRFMk9vb2VDQzJmd05SaU43Nk1lK3loWnJqRzRuOG5zQWp4M0QzbGVIeXJBeG41aE9xTkciLCJtYWMiOiIxY2QwOGMwMDMzNmIwOGFjMzRkNGU4YzEwZDdmNWE3Y2NkNGE1YzljMDc3ZWY2YzFhODJhN2M1MmRlNmZiMjk2In0=; _gat_gtag_UA_145833656_1=1; _gat_UA-145833656-1=1',
}

kfc_data = {
    'cname': '',
    'pid': '',
    'keyword': '西安',
    'pageIndex': '1',
    'pageSize': '10',
}

In [24]:
json.dumps(kfc_data)

'{"cname": "", "pid": "", "keyword": "\\u897f\\u5b89", "pageIndex": "1", "pageSize": "10"}'

In [27]:
response = requests.post(kfc, data=kfc_data, headers=header)
print(response.status_code)

200


In [33]:
for loc in json.loads(response.text)['Table1']:
    print(loc)

{'rownum': 1, 'storeName': '东大街（西安）', 'addressDetail': '东大街53号', 'pro': '24小时,24小时,Wi-Fi,Wi-Fi,礼品卡', 'provinceName': '青海省', 'cityName': '西宁市'}
{'rownum': 2, 'storeName': '同安', 'addressDetail': '同安区西安路西安广场一层二层', 'pro': '24小时,24小时,Wi-Fi,Wi-Fi,点唱机,点唱机,礼品卡', 'provinceName': '福建省', 'cityName': '厦门市'}
{'rownum': 3, 'storeName': '贺兰山（西安）', 'addressDetail': '游艺东街6号一层', 'pro': 'Wi-Fi,Wi-Fi,店内参观,店内参观,礼品卡', 'provinceName': '宁夏', 'cityName': '石嘴山市'}
{'rownum': 4, 'storeName': '西安明德门', 'addressDetail': '朱雀大街58号金水大厦一、二层', 'pro': 'Wi-Fi,Wi-Fi,店内参观,店内参观,礼品卡,手机点餐,手机点餐', 'provinceName': '陕西省', 'cityName': '西安市'}
{'rownum': 5, 'storeName': '西安兴庆', 'addressDetail': '兴庆路28号华润万家超一二层肯德基', 'pro': 'Wi-Fi,Wi-Fi,店内参观,店内参观,礼品卡', 'provinceName': '陕西省', 'cityName': '西安市'}
{'rownum': 6, 'storeName': '西安莲湖', 'addressDetail': '莲湖路25#汉庭酒店一二层', 'pro': '24小时,24小时,Wi-Fi,Wi-Fi,店内参观,店内参观,礼品卡', 'provinceName': '陕西省', 'cityName': '西安市'}
{'rownum': 7, 'storeName': '西安文景', 'addressDetail': '凤城五路与文景路十字东南角人人乐超一楼', 'pro': 'Wi-Fi,W